# PCA Plots

## General settings and loading files

In [1]:
%%time

% matplotlib notebook
% load_ext autoreload
% autoreload 2

% run general_settings.py
# ^ Takes ~20s to run if it finds the dumpfiles
# ^ WARNING: It will hog ~2.02 Gb RAM


[00:37:40] 'panels' dict
[00:37:40] 'galanter', 'present', 'missing' dataframes
[00:37:40] 'panel_labels'
[00:37:40] 'panel_names' dict
[00:37:40] 'panel_rsIDs' dict
[00:37:40] 'genome' dataframe
[00:37:47] 'lat' dataframe
[00:38:00] 'control_genotypes' huge datagrame
[00:38:00] 'control_rsIDs' dict to filter it ^
[00:38:00] 'cp_factors' list
[00:38:00] 'control_names' dict
[00:38:00] 'df_1000G_samples'
[00:38:00] 'df_1000G_SNPs'
[00:38:00] 'df_1000G_genotypes'
[00:38:00] 'df_1000G_populations'
[00:38:00] 'df_1000G_genotypes_alleles'

/home/juan/repos/tesina/settings/genome.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  centromers["chromosome"] = [i.replace("CEN", "") for i in centromers.index]
/home/juan/anaconda3/lib/python3.5/site-packages/pandas/io/parsers.py:1572: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())



[00:38:01] 'mafs' dataframe
CPU times: user 16.7 s, sys: 3.29 s, total: 20 s
Wall time: 21.7 s


/home/juan/anaconda3/lib/python3.5/site-packages/pandas/io/parsers.py:1574: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())


## Create the datasets
Lists of sample IDs of chosen populations to later filter 1000 Genomes data.

In [23]:
% run datasets_settings.py

! echo; wc -l /home/juan/tesina/dataset_dumps/*.samples

[00:59:51] 'dataset_names'
[00:59:51] 'dataset_populations'
[00:59:51] 'dataset_samples' created and written to files

  346 /home/juan/tesina/dataset_dumps/L.samples
  437 /home/juan/tesina/dataset_dumps/LE.samples
  644 /home/juan/tesina/dataset_dumps/LEA.samples
  852 /home/juan/tesina/dataset_dumps/LEAC.samples
  955 /home/juan/tesina/dataset_dumps/LEACI.samples
 3234 total


## PCA plots

In [ ]:
%%time

# == WARNING ==
# This cell will take ~4m to execute
# And it will take ~500 Mb RAM, I think

from plotters.pca import PCAPlotter

pca_plotter = PCAPlotter()
components_to_compare = [(0, 1)]

def make_filename(dataset_label, panel_labels, components_to_compare):
    return "{}_{}_PCx{}".format(dataset_label, "_".join(panel_labels).lower(),
                                len(components_to_compare) * 2)

for dataset_label, sample_ids in dataset_samples.items():
    fig_title = "Dataset: {}".format(dataset_names[dataset_label])
    
    # GAL Panles
    dataset_genotypes = df_1000G_genotypes.loc[sample_ids, :]
    filename = make_filename(dataset_label, panel_labels, components_to_compare)
    pca_plotter.plot(fig_title, panel_rsIDs, dataset_genotypes, df_1000G_samples,
                     components_to_compare, panel_names, filename)
    
    # Control Panels
    dataset_genotypes = control_genotypes.loc[sample_ids, :]
    filename = make_filename(dataset_label, control_labels, components_to_compare)
    pca_plotter.plot(fig_title, control_rsIDs, dataset_genotypes, df_1000G_samples,
                     components_to_compare, control_names, filename)
    
    # Control Panels - Extra components
    only_big_cp = {"100": control_rsIDs["100"]}
    more_components_to_compare = [(2, 3), (4, 5), (6, 7)]
    dataset_genotypes = control_genotypes.loc[sample_ids, :]
    filename = make_filename(dataset_label, ["100"], more_components_to_compare)
    pca_plotter.plot(fig_title, only_big_cp, dataset_genotypes, df_1000G_samples,
                     more_components_to_compare, control_names, filename)